In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import matplotlib.pyplot as plt

In [2]:
xgb_train = pd.read_csv('./xgb_stack/xgb_layer_1_train.csv')
xgb_val = pd.read_csv('./xgb_stack/xgb_layer_1_val.csv')
# fm_train = pd.read_csv('./fm_stack/fm_layer_1_train.csv')
# fm_val = pd.read_csv('./fm_stack/fm_layer_1_val.csv')
rf_train = pd.read_csv('./rf_stack/rf_layer_1_train.csv')
rf_val = pd.read_csv('./rf_stack/rf_layer_1_val.csv')

In [3]:
fold1_index = pd.read_csv('./index/fold1_index.csv')
fold2_index = pd.read_csv('./index/fold2_index.csv')
validation_index = pd.read_csv('./index/validation_index.csv')
y_train = pd.read_csv('../kaggle-quora/features/y_train_oversampled.csv')

y_train_fold1 = y_train.loc[fold1_index.values.flatten(), :]
y_train_fold2 = y_train.loc[fold2_index.values.flatten(), :]
y_validation_fold = y_train.loc[validation_index.values.flatten(), :]

In [4]:
X_train_stack = pd.concat([xgb_train, rf_train], axis=1)
X_val_stack = pd.concat([xgb_val, rf_val], axis=1)
X_train_stack.head()

,xgb_layer_1_train,rf_layer_1_train
0,0.266854,0.260000
1,0.381091,0.346667
2,0.245272,0.240000
3,0.762464,0.493333
4,0.219644,0.206667


In [5]:
y_train_stack = pd.concat([y_train_fold1, y_train_fold2])

In [6]:
print(X_train_stack.shape)
print(X_val_stack.shape)
print(y_train_stack.shape)

(520325, 2)
(260161, 2)
(520325, 1)


In [10]:
logit = LogisticRegression(C=1000)
logit.fit(X_train_stack, y_train_stack['0'])
logit_val = logit.predict_proba(X_val_stack)
print(log_loss(y_train_stack['0'], pd.DataFrame(logit.predict_proba(X_train_stack))[1]))
print(log_loss(y_validation_fold, pd.DataFrame(logit_val)[1]))

#    1: 0.20114846208830159
#   10: 0.20113978094252283
#  100: 0.20113891529690833
# 1000: 0.20113882875709824
#10000: 0.20113882010336498

0.199719662581
0.211932018387


In [11]:
xgb_x_test = pd.read_csv('./submission/xgb_jaccard_magic_seed12357_n315.csv')['is_duplicate']
rf_x_test = pd.read_csv('./submission/submission_rf_layer_1_test.csv')['is_duplicate']
# fm_x_test = pd.read_csv('./submission/fm_pred_jacad_magic.csv')['is_duplicate']
X_test_stack = pd.concat((xgb_x_test, rf_x_test), axis=1)

In [12]:
logit_pred = logit.predict_proba(X_test_stack)
(logit_pred.shape)

(2345796, 2)

In [15]:
submission = pd.DataFrame()
submission['test_id'] = X_test_stack.index.values
submission['is_duplicate'] = pd.DataFrame(logit_pred)[1]
submission.to_csv('stack_xgb_rf_prev_test.csv', index=False)

In [18]:
submission = pd.read_csv('./stack_xgb_rf_prev_test.csv')
submission.head()

,test_id,is_duplicate
0,0,0.010460
1,1,0.984471
2,2,0.074735
3,3,0.006864
4,4,0.863128
